<a href="https://colab.research.google.com/github/alesaccoia/IULM_DDM2324_Notebooks/blob/main/33_Retrieval_Augmented_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Search con OpenAI

In [ ]:
import csv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import chromadb

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR API KEY" # https://platform.openai.com/api-keys


def get_name(_):
    return "Test"


## Carichiamo il dataset

Carichiamo la lista degli attori, per la "fonte" del documento usiamo un URL

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="actors_description.csv", source_column="fonte")
docs = loader.load()

In [ ]:
# Visualizziamo un articolo
print(docs[3])

page_content='descrizione_persona: Aisha Patel, Indian actress acclaimed for her roles in Bollywood musicals\nfonte: http://www.bollywood-stars.in/aisha-patel' metadata={'source': 'http://www.bollywood-stars.in/aisha-patel', 'row': 3}


## Utilizzo degli embeddings di OpenAI

E creazione di un "vector store" (Attenzione l'API é a Pagamento)

In [ ]:
embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(
    docs, embeddings, collection_name="attori"
)

## Ricerca dal Vector Database

In [ ]:
query = "Chi é Sophie Lephebre?"
retrieved_docs = vector_store.similarity_search(query)
print(retrieved_docs[0].page_content)

descrizione_persona: Sophie Lefèvre, actrice française célèbre pour ses rôles dans des films romantiques
fonte: http://www.cinema-francais.com/sophie-lefevre


## Utilizzo in un chatbot


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Create the retriever using the vector store
retriever = vector_store.as_retriever()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)
chain = ConversationalRetrievalChain.from_llm(model, retriever= retriever, memory= memory)

In [ ]:
chain.run({'question': 'Chi é Sophie Lephebre?'})

"Sophie Lefèvre è un'attrice francese famosa per i suoi ruoli nei film romantici."

In [ ]:
chain.run({'question': 'Di che nazionalitá é?'})

'Sophie Lefèvre è di nazionalità francese.'